In [2]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow as tf
import subprocess
import os
import pandas as pd
import random
import shutil
import numpy as np
import json
import re
import pathlib

from langdetect import detect
from google.colab import drive, files #if use colab
from tensorflow.nn import relu, tanh, softmax
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.lite.python import interpreter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
# if use colab
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#if use colab
git_dir = "/content/IOH-Chat-App"
git_url = "https://github.com/bangkit-team/IOH-chat-app.git"

if not os.path.exists(git_dir):
  subprocess.call(["git", "clone", git_url])

In [6]:
filedir1 = "/content/IOH-chat-app/MachineLearning/datasets/translation/result/eng-ind.csv" # #if use colab
# filedir1 = "../../datasets/translate sentence/result/eng-ind.csv" #if use local env

In [7]:
df = pd.read_csv(filedir1)
df

,English,Indonesia
0,Run!,Lari!
1,Who?,Siapa?
2,Wow!,Wow!
3,Help!,Tolong!
4,Jump!,Lompat!
...,...,...
15359,Limitation of this capability causes opportuni...,Keterbatasan kemampuan ini menyebabkan tertutu...
15360,Subjective approach evaluates poverty based on...,Pendekatan subyektif menilai kemiskinan berdas...
15361,"Limited sufficiency and food quality , seen fr...","terbatasnya kecukupan dan mutu pangan , diliha..."
15362,Around 20 percents people with the lowest inco...,Sekitar 20 persen penduduk dengan tingkat pend...


In [8]:
start_mark = '<start>'
end_mark = '<end>'

In [9]:
class TranslatorDataset:
  def __init__(self, dataframe):
    self.dataframe = dataframe
    self.input_tokenizer = None
    self.target_tokenizer = None
    self._load_data_from_file()

  def _load_data_from_file(self):
    df = self.dataframe

    input_lang = df.English.values
    target_lang = df.Indonesia.values

    return input_lang, target_lang

  def _normalize_and_preprocess(self, text, use_mark=False):
    if use_mark:
      text = re.sub(r"[^\w\s]","",text)
      text = text.strip()
      text = " ".join([start_mark, text, end_mark])
    else:
      text = re.sub(r"[^\w\s]","",text)
      text = text.strip()

    return text

  def _tokenize(self, sentences, num_words, maxlen):
    tokenizer = layers.TextVectorization(
        max_tokens=num_words, standardize="lower", output_sequence_length=maxlen)
    tokenizer.adapt(sentences)
    sequences = tokenizer(sentences)

    return sequences, tokenizer

  def _create_dataset(self):
    input_lang, target_lang = self._load_data_from_file()

    input_sentence = np.array(
        list(map(lambda x: self._normalize_and_preprocess(x, False), input_lang)))
    
    target_sentence = np.array(
        list(map(lambda y: self._normalize_and_preprocess(y, True), target_lang)))
    
    return input_sentence, target_sentence

  def _load_dataset(self, num_words):
    input_lang, target_lang = self._create_dataset()

    self.maxlen = 15
    self.buffer_size = len(input_lang)

    input_sequences, input_tokenizer = self._tokenize(
        input_lang, num_words, self.maxlen)
    
    target_sequences, target_tokenizer = self._tokenize(
        target_lang, num_words, self.maxlen,)

    return (input_sequences, input_tokenizer), (target_sequences, target_tokenizer)
  
  def get(self, num_words, batch_size):
    input, target = self._load_dataset(num_words)

    input_sequences, self.input_tokenizer = input
    target_sequences, self.target_tokenizer = target

    dataset = tf.data.Dataset.from_tensor_slices((input_sequences, target_sequences))
    dataset = dataset.shuffle(self.buffer_size).batch(batch_size, drop_remainder=True)
    dataset = dataset.cache().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return self.input_tokenizer, self.target_tokenizer, dataset

In [10]:
num_words = 20000
batch_size = 64

In [11]:
translator_dataset = TranslatorDataset(df)

input_tokenizer, target_tokenizer, dataset = translator_dataset.get(num_words, batch_size)

In [12]:
input_batch, target_batch = next(iter(dataset))

In [13]:
input_batch.shape, target_batch.shape

(TensorShape([64, 15]), TensorShape([64, 15]))

In [14]:
input_vocab_size = input_tokenizer.vocabulary_size()
target_vocab_size = target_tokenizer.vocabulary_size()
input_maxlen = input_batch.shape[1]
target_maxlen = target_batch.shape[1]

input_maxlen, target_maxlen, input_vocab_size, target_vocab_size

(15, 15, 13731, 15293)

In [15]:
# input_wi_json = "input_word_index.json"

# with open(input_wi_json, 'w') as f:
#     json.dump(input_tokenizer.word_index, f)

# # files.download(input_wi_json)

In [16]:
# target_wi_json = "target_index_word.json"

# with open(target_wi_json, 'w') as f:
#     json.dump(target_tokenizer.index_word, f)

# # files.download(target_wi_json)

In [17]:
input_example = input_batch[-1]
input_example

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([  16,    9,  120,   84,   27,  393,  483,   70,   61,  357,    2,
        446,    4, 2718,  372])>

In [18]:
target_example = target_batch[-1]
target_example

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([   2, 1270,   63,  149,   10,  268,  446,   73,   78, 1301, 1608,
       1272,  341,    5, 2792])>

In [19]:
input_vocab = np.array(input_tokenizer.get_vocabulary())
sequence = input_vocab[input_example.numpy()]
" ".join(sequence)

'that is micro finance with rp 1 million can change the life of mrs sri'

In [20]:
target_vocab = np.array(target_tokenizer.get_vocabulary())
sequence = target_vocab[target_example.numpy()]
" ".join(sequence)

'<start> itulah keuangan mikro dengan rp 1 juta dapat mengubah kehidupan ibu sri dan keluarganya'

In [21]:
embed_dims = 512
units = 1024

In [22]:
class Encoder():
  def __init__(self, input_vocab_size, embedding_dims, units):
    self.units = units
    self.batch_size = batch_size
    self.input_vocab_size = input_vocab_size
    self.embedding_dims = embedding_dims

    self.embedding = layers.Embedding(self.input_vocab_size, self.embedding_dims)
    self.lstm_layer = layers.LSTM(self.units,
                                 return_sequences=True,
                                 return_state=True,
                                 recurrent_initializer='glorot_uniform')

  def call(self, inputs):
    embedding = self.embedding(inputs)
    encoder = self.lstm_layer(embedding)

    return encoder

In [23]:
class BahdanauAttention(layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.w1 = layers.Dense(units, use_bias=True) 
    self.w2 = layers.Dense(units, use_bias=True) 
    self.attention = tf.keras.layers.AdditiveAttention()

  def call(self, query, values):
    w1_query = self.w1(query)
    w2_key = self.w2(values)
    query_mask = tf.ones(tf.shape(query)[:-1], dtype=bool)

    context_vector, attention_weights = self.attention(
        inputs = [w1_query, values, w2_key],
        return_attention_scores = True)

    return context_vector, attention_weights

In [42]:
class Decoder:
  def __init__(self, output_vocab_size, embedding_dims, units):
    self.units = units
    self.output_vocab_size = output_vocab_size
    self.embedding_dims = embedding_dims

    self.embedding = layers.Embedding(self.output_vocab_size, self.embedding_dims)
    self.lstm_layer = layers.LSTM(self.units,
                                  return_sequences=True,
                                  return_state=True,
                                  recurrent_initializer='glorot_uniform')
    self.attention = BahdanauAttention(self.units)
    self.dense1 = layers.Dense(self.units, activation=tanh, use_bias=False)
    self.dropout = layers.Dropout(.5)
    self.dense2 = layers.TimeDistributed(layers.Dense(self.output_vocab_size))

  def call(self, inputs, en_outputs, state):
    embedding = self.embedding(inputs)
    dec_outputs, dec_h_state, dec_c_state = self.lstm_layer(
        embedding, initial_state=state)
    
    context_vector, attention_weights = self.attention(
        query=dec_outputs, values=en_outputs)
    
    context_and_rnn_output = tf.concat([context_vector, dec_outputs], axis=-1)

    attention_vector = self.dense1(context_and_rnn_output)
    outputs = self.dropout(attention_vector)
    outputs = self.dense2(outputs)

    return outputs, attention_weights

In [43]:
encoder = Encoder(input_vocab_size, embed_dims, units)

en_outputs, en_h_state, en_c_state = encoder.call(input_batch)

en_outputs.shape, en_h_state.shape, en_c_state.shape

(TensorShape([64, 15, 1024]), TensorShape([64, 1024]), TensorShape([64, 1024]))

In [44]:
decoder = Decoder(target_vocab_size, embed_dims, units)
dec_outputs= decoder.call(target_batch, en_outputs, [en_h_state, en_c_state])

dec_outputs.shape

AttributeError: ignored

In [45]:
lr = 0.001
epochs = 30

optimizer = tf.keras.optimizers.Adam(
    learning_rate=lr, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
loss = tf.keras.losses.SparseCategoricalCrossentropy(
       from_logits=True, reduction='none')

In [46]:
class TranslatorModel:
  def __init__(self, input_vocab_size, 
               target_vocab_size, 
               embed_dims, 
               units):
    self.input_vocab_size = input_vocab_size
    self.target_vocab_size = target_vocab_size
    self.embed_dims = embed_dims
    self.units = units

    self.encoder = Encoder(
        self.input_vocab_size, self.embed_dims, self.units)
    
    self.decoder = Decoder(
        self.target_vocab_size, self.embed_dims, self.units)
  
  def build_model(self):
    en_inputs = layers.Input(shape=(None,))

    en_output, en_h_state, en_c_state = self.encoder.call(en_inputs)

    dec_outputs = self.decoder.call(
        en_inputs, en_output, [en_h_state, en_c_state])

    model = Model(inputs=[en_inputs], 
                  outputs=[dec_outputs])
    return model

In [47]:
translator_model = TranslatorModel(
    input_vocab_size,
    target_vocab_size,
    embed_dims,
    units,
)
model = translator_model.build_model()

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=["accuracy"]
)

In [48]:
checkpoint_path = "checkpoint/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

callback_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=3, 
    verbose=1)

callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor='loss', 
    verbose=1, 
    save_weights_only=True, 
    save_best_only=True)

callbacks = [callback_early_stopping,
             callback_checkpoint]

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, None, 512)    7030272     ['input_2[0][0]']                
                                                                                                  
 embedding_7 (Embedding)        (None, None, 512)    7830016     ['input_2[0][0]']                
                                                                                                  
 lstm_6 (LSTM)                  [(None, None, 1024)  6295552     ['embedding_6[0][0]']            
                                , (None, 1024),                                             

In [ ]:
model.fit(dataset,
          epochs=epochs,
          callbacks=callbacks,
          verbose=1)

Epoch 1/30
240/240 [==============================] - ETA: 0s - loss: nan - time_distributed_3_loss: nan - bahdanau_attention_3_loss: nan - time_distributed_3_accuracy: 0.3104 - bahdanau_attention_3_accuracy: 0.3104
Epoch 1: loss did not improve from inf
240/240 [==============================] - 19s 80ms/step - loss: nan - time_distributed_3_loss: nan - bahdanau_attention_3_loss: nan - time_distributed_3_accuracy: 0.3104 - bahdanau_attention_3_accuracy: 0.3104
Epoch 2/30
240/240 [==============================] - ETA: 0s - loss: nan - time_distributed_3_loss: nan - bahdanau_attention_3_loss: nan - time_distributed_3_accuracy: 0.3104 - bahdanau_attention_3_accuracy: 0.3104
Epoch 2: loss did not improve from inf
240/240 [==============================] - 19s 77ms/step - loss: nan - time_distributed_3_loss: nan - bahdanau_attention_3_loss: nan - time_distributed_3_accuracy: 0.3104 - bahdanau_attention_3_accuracy: 0.3104
Epoch 3/30
139/240 [================>.............] - ETA: 7s - loss

In [ ]:
# if use colab
saved_model_path = "/content/drive/MyDrive/Company Case Bangkit/TranslationModel/saved_model"

# if use local env
# saved_model_path = "code/translate sentence/saved_model"
saved_model_dir = os.path.dirname(saved_model_path)

if os.path.exists(saved_model_dir):
  shutil.rmtree(saved_model_dir)
  
model.save(saved_model_path)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS,
  tf.lite.OpsSet.SELECT_TF_OPS
]
converter.experimental_new_converter = True
converter.allow_custom_ops = True

tflite_model = converter.convert()

tflite_model_file = pathlib.Path('translation.tflite')
tflite_model_file.write_bytes(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter("translation.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_details, output_details

In [ ]:
class Translator(tf.Module):
  def __init__(self, encoder, decoder, input_tokenizer, target_tokenizer):
    self.encoder = encoder
    self.decoder = decoder
    self.input_tokenizer = input_tokenizer
    self.target_tokenizer = target_tokenizer

    self.start_token = target_tokenizer("<start>")
    self.end_token = target_tokenizer("<end>")

    self.target_word_index = (
        tf.keras.layers.StringLookup(
            vocabulary=target_tokenizer.get_vocabulary(),
            mask_token='',
            invert=True))
    
    index_from_string = tf.keras.layers.StringLookup(
        vocabulary=target_tokenizer.get_vocabulary(), mask_token='')

    self.start_token = index_from_string(tf.constant("<start>"))
    self.end_token = index_from_string(tf.constant("<end>"))

  def _sample(self, logits, temperature):
    if temperature == 0.0:
      new_sequences = tf.argmax(logits, axis=-1)
    else: 
      logits = tf.squeeze(logits, axis=1)
      new_sequences = tf.random.categorical(logits/temperature,
                                          num_samples=1)

    return new_sequences

  def _tokens_to_text(self, sequence):
    result_sequence = self.target_word_index(sequence)

    result_text = tf.strings.reduce_join(result_sequence,
                                        axis=1, separator=' ')

    result_text = tf.strings.strip(result_text)
    return result_text
    
  def _translate(self,
                input, *,
                max_length=50,
                return_attention=True,
                temperature=1.0):
    batch_size = tf.shape(input)[0]

    input_tokens = self.input_tokenizer(input)
    en_output, en_h_state, en_c_state = self.encoder.call(input_tokens)

    dec_state = [en_h_state, en_c_state]
    new_sequences = tf.fill([batch_size, 1], self.start_token)

    result_sequences = tf.TensorArray(tf.int64, size=1, dynamic_size=True)
    attention = tf.TensorArray(tf.float32, size=1, dynamic_size=True)
    done = tf.zeros([batch_size, 1], dtype=tf.bool)

    for t in tf.range(max_length):
      dec_outputs, attention_weights = self.decoder.call(
          new_sequences, en_output, dec_state)
      attention = attention.write(t, attention_weights)
      new_sequences = self._sample(dec_outputs, temperature)

      done = done | (new_sequences == self.end_token)
      new_sequences = tf.where(done, tf.constant(0, dtype=tf.int64), new_sequences)

      result_sequences = result_sequences.write(t, new_sequences)

      if tf.reduce_all(done):
        break

    result_sequences = result_sequences.stack()
    result_sequences = tf.squeeze(result_sequences, -1)
    result_sequences = tf.transpose(result_sequences, [1, 0])

    result_text = self._tokens_to_text(result_sequences)

    if return_attention:
      attention_stack = attention.stack()
      attention_stack = tf.squeeze(attention_stack, 2)
      attention_stack = tf.transpose(attention_stack, [1, 0, 2])

      return {'text': result_text, 'attention': attention_stack}
    else:
      return {'text': result_text}

  def lang_detector(self, sentence):
    return detect(sentence)

  @tf.function(input_signature=[tf.TensorSpec(dtype=tf.string, shape=[None])])
  def tf_translate(self, input):
    return self._translate(input)

In [ ]:
input_wi = "/content/input_word_index.json"
target_iw = "/content/target_index_word.json"

translator = Translator(
    encoder,
    decoder,
    input_tokenizer, 
    target_tokenizer,
)

In [ ]:
text_input = tf.constant(["She took it off."])
lang_detector = translator.lang_detector(text_input[0].numpy().decode())

if lang_detector == "en":
  translate = translator.tf_translate(text_input)
  print(translate["text"][0].numpy().decode())
else:
  print("Bahasa tidak dikenali")